<a href="https://colab.research.google.com/github/Husseinalia99/My-graduation-project/blob/main/efficientnetb5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install --root-user-action=ignore
!pip install pandas

!pip install split_folders

ERROR: You must give at least one requirement to install (see "pip help install")
^C


In [ ]:
!ls ../input/skin-diseases-image-dataset/IMG_CLASSES

'1. Eczema 1677'
'10. Warts Molluscum and other Viral Infections - 2103'
'2. Melanoma 15.75k'
'3. Atopic Dermatitis - 1.25k'
'4. Basal Cell Carcinoma (BCC) 3323'
'5. Melanocytic Nevi (NV) - 7970'
'6. Benign Keratosis-like Lesions (BKL) 2624'
'7. Psoriasis pictures Lichen Planus and related diseases - 2k'
'8. Seborrheic Keratoses and other Benign Tumors - 1.8k'
'9. Tinea Ringworm Candidiasis and other Fungal Infections - 1.7k'


In [ ]:
import splitfolders
import os

os.makedirs('output')
os.makedirs('output/train')
os.makedirs('output/val')
os.makedirs('output/test')

loc = "../input/skin-diseases-image-dataset/IMG_CLASSES/"

splitfolders.ratio(loc,output = "output",seed = 42,ratio = (0.80,.1,.1))

ModuleNotFoundError: No module named 'splitfolders'

> In kaggle we need to create a directory for our data after splitting but if we do this using jupyter notebook on our PC/laptop,we can just specify the input path and output path.

In [ ]:
import os
for dirpath,dirname,filename in os.walk("./output"):
    print(f"There are {len(dirname)} and {len(filename)} in '{dirpath}'.")

> After modifying our input data and before the start of modelling its always best to visualize some random images of the dataset

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mping
import random

def plot_random_image(target_dir,target_class):
    target_folder = target_dir + target_class
    random_image = random.sample(os.listdir(target_folder),1)
    img = mping.imread(target_folder + "/" + random_image[0])
    plt.imshow(img)
    plt.title(target_class)
    plt.axis("off");
    return img


In [ ]:
fig = plt.figure(figsize=(10, 7))
fig.add_subplot(2,2,1)
img_1 = plot_random_image(target_dir = "./output/test/",target_class = "2. Melanoma 15.75k")
fig.add_subplot(2,2,2)
img_2 = plot_random_image(target_dir = "./output/test/",target_class = "4. Basal Cell Carcinoma (BCC) 3323")
fig.add_subplot(2,2,3)
img_3 = plot_random_image(target_dir = "./output/test/",target_class = "5. Melanocytic Nevi (NV) - 7970")
fig.add_subplot(2,2,4)
img4 = plot_random_image(target_dir = "./output/test/",target_class = "1. Eczema 1677")

# ***Modelling***

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

> When we use mixed_precision training the computation speed is increased by 3x times based on the GPU available. Mixed precision enables training using float16 half-precision variables whenever possible.

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

train_dir = "./output/train"
test_dir =  "./output/test"
val_dir = "./output/val"

train_data = image_dataset_from_directory(train_dir,label_mode = "categorical",
                                          image_size = (224,224),batch_size = 32,
                                         shuffle = True,seed = 42)
test_data = image_dataset_from_directory(test_dir,label_mode = "categorical",
                                          image_size = (224,224),batch_size = 32,
                                         shuffle = False,seed = 42)
val_data = image_dataset_from_directory(val_dir,label_mode = "categorical",
                                          image_size = (224,224),batch_size = 32,
                                         shuffle = False,seed = 42)

> image_dataset_from_directory() imports and converts our input data into tf.data.Dataset format and it is generally faster than ImageDataGenerator().

In [ ]:
class_names = train_data.class_names
print(len(class_names))
print(class_names)

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor = "val_loss",patience = 6,
                                             min_delta = 0.0001)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = "val_loss",factor = 0.2,
                                                patience = 4,min_lr = 1e-7)

> * Earlystopping callback stops training when the model stops improving in terms of validation loss(in this case) and prevents overfitting.
> * ReduceLROnPlateau reduces the learning rate by 5x(in this case) whenever validation loss is not improving.
> * With the combination of Earlystopping and ReduceLROnPlateau callback when can train our model for any number of epochs without worrying about overfitting.


In [ ]:
train_data = train_data.prefetch(buffer_size = tf.data.AUTOTUNE)
test_data = test_data.prefetch(buffer_size = tf.data.AUTOTUNE)
val_data = val_data.prefetch(buffer_size = tf.data.AUTOTUNE)

> Data is prefetched to reduce computation time.

In [ ]:
base_model = tf.keras.applications.EfficientNetB5(include_top = False)
base_model.trainable = False

> First we are going to be training feature extractor EfficientNetB5 model. Feature extractor transfer learning involves using the pretrained weights of a model trained on another dataset similar to own for our own problem. Here the output layer of pretrained model is modified according our own problem.

In [ ]:
for layer_num,layer in enumerate(base_model.layers):
    print(layer_num,layer.name,layer.trainable)

> As we can see EfficientNetB5 consists of 575 layers without including the output layer and the most important thing to note among these layers is the rescaling layer present right after the input layer,this means that we dont have to rescale our data during preprocessing.

In [ ]:
from tensorflow.keras.layers.experimental import preprocessing

data_aug = tf.keras.Sequential([
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomFlip("horizontal")
],name = "data_augmentation_layer")

> Data augmentation is used here to prevent overfitting, we can experiment without data augmentation and check whether the model overfits or not,but since we are using a transfer learning Architecture such as EfficientNet,its best to include data augmentation since the probability of our model overfitting is very high.

In [ ]:
inputs = layers.Input(shape = (224,224,3),name = "input_layer")
x = data_aug(inputs)
x = base_model(x)
x = layers.GlobalAvgPool2D(name = "pooling_layer")(x)
x = layers.Dense(32,activation = "relu",kernel_initializer = tf.keras.initializers.he_normal())(x)
x = layers.Dense(10)(x)
outputs = layers.Activation("softmax",dtype = tf.float32)(x)
model = tf.keras.Model(inputs,outputs)

In [ ]:
model.summary()

In [ ]:
for layer_num,layer in enumerate(model.layers):
    print(layer_num,layer.name,layer.trainable,layer.dtype,layer.dtype_policy)

> We can clearly see here that mixed_precision policy is implemented and our EfficientNetB5 model is completely frozen. Now we can compile and fit our model.

In [ ]:
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(),optimizer = tf.keras.optimizers.Adam(),
             metrics = ["accuracy"])

In [ ]:
history_1 = model.fit(train_data,epochs = 15,steps_per_epoch = len(train_data),
                     validation_data = val_data,validation_steps = int(0.25*len(val_data)),
                     callbacks = [early_stop,reduce_lr])

In [ ]:
print("Validation Accuracy",model.evaluate(val_data))
print("Testing Accuracy",model.evaluate(test_data))

In [ ]:
def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.
  """
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

In [ ]:
plot_loss_curves(history_1)

> * Training Accuracy - 74.4%
> * Testing Accuracy  - 71.2%
> * Validation Accuracy - 71.7%

> Note: Specified number of epochs as 15 but training stopped at 12 and it wasn't beacuse of earlystopping callback,not sure why it stopped early, if you got any ideas please do mention it.

# Fine-Tuned EfficientNetB5

In [ ]:
base_model.trainable = True

for layer in base_model.layers[:-30]:
    layer.trainable = False

> Now in order to improve our model's performance we unfreeze the top 30 layers closer to the output layer and let them train on our data instead of using pre-trained weights.

In [ ]:
for layer_num,layer in enumerate(model.layers):
    print(layer_num,layer.name,layer.trainable,layer.dtype_policy)

In [ ]:
for layer_num,layer in enumerate(base_model.layers):
    print(layer_num,layer.name,layer.trainable)

In [ ]:
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(),optimizer = tf.keras.optimizers.Adam(1e-4),
             metrics = ["accuracy"])

In [ ]:
history_2 = model.fit(train_data,epochs = 30,steps_per_epoch = len(train_data),
                     initial_epoch = history_1.epoch[-1],
                     validation_data = val_data,validation_steps = int(0.25*len(val_data)),
                     callbacks = [early_stop,reduce_lr])

In [ ]:
print("Validation Accuracy",model.evaluate(val_data))
print("Testing Accuracy",model.evaluate(test_data))

In [ ]:
def compare_historys(original_history, new_history, initial_epochs):
    """
    Compares two model history objects.
    """
    # Get original history measurements
    acc = original_history.history["accuracy"]
    loss = original_history.history["loss"]

    print(len(acc))

    val_acc = original_history.history["val_accuracy"]
    val_loss = original_history.history["val_loss"]

    # Combine original history with new history
    total_acc = acc + new_history.history["accuracy"]
    total_loss = loss + new_history.history["loss"]

    total_val_acc = val_acc + new_history.history["val_accuracy"]
    total_val_loss = val_loss + new_history.history["val_loss"]

    print(len(total_acc))
    print(total_acc)

    # Make plots
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(total_acc, label='Training Accuracy')
    plt.plot(total_val_acc, label='Validation Accuracy')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(total_loss, label='Training Loss')
    plt.plot(total_val_loss, label='Validation Loss')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

In [ ]:
compare_historys(history_1,history_2,initial_epochs = 12)

> *  Training accuracy - 85.58%
> *  Testing Accuracy - 77.08%
> *  Validation Accuracy - 76.02%

# Model Evalutation

In [ ]:
pred_probs = model.predict(test_data)
pred_probs[0]

In [ ]:
pred_classes = pred_probs.argmax(axis =1)
print(pred_classes[0])
print(class_names[pred_classes[0]])

In [ ]:
y_labels = []
for image,label in test_data.unbatch():
    y_labels.append(label.numpy().argmax())
y_labels[:20]

In [ ]:
print(len(pred_classes))
print(len(y_labels))

In [ ]:
from sklearn.metrics import classification_report
print("Classification report\n",classification_report(y_labels,pred_classes))

In [ ]:
classification_dict = classification_report(y_labels,pred_classes,output_dict = True)
classification_dict

In [ ]:
classification_f1_scores = {}
for k,v in classification_dict.items():
    if k == "accuracy":
        break
    else:
        classification_f1_scores[class_names[int(k)]] = v["f1-score"]
classification_f1_scores

In [ ]:
f1_scores = pd.DataFrame({"class_name":list(classification_f1_scores.keys()),
                         "F1-Scores":list(classification_f1_scores.values())})
f1_scores.sort_values("F1-Scores",ascending = False)

> From the F1-Scores dataframe we can clearly see that our model perform best on Melanocytic Nevi with an F1-Score of 0.92 and perform the worst on Atopic Dematitis.

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False):
  """Makes a labelled confusion matrix comparing predictions and ground truth labels.

  If classes is passed, confusion matrix will be labelled, if not, integer class values
  will be used.

  Args:
    y_true: Array of truth labels (must be same shape as y_pred).
    y_pred: Array of predicted labels (must be same shape as y_true).
    classes: Array of class labels (e.g. string form). If `None`, integer labels are used.
    figsize: Size of output figure (default=(10, 10)).
    text_size: Size of output figure text (default=15).
    norm: normalize values or not (default=False).
    savefig: save confusion matrix to file (default=False).

  Returns:
    A labelled confusion matrix plot comparing y_true and y_pred.

  Example usage:
    make_confusion_matrix(y_true=test_labels, # ground truth test labels
                          y_pred=y_preds, # predicted labels
                          classes=class_names, # array of class label names
                          figsize=(15, 15),
                          text_size=10)
  """
  # Create the confustion matrix
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
  fig.colorbar(cax)

  # Are there a list of classes?
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])

  # Label the axes
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes),
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)

  # Make x-axis labels appear on bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  ### Added: Rotate xticks for readability & increase font size (required due to such a large confusion matrix)
  plt.xticks(rotation=70, fontsize=text_size)
  plt.yticks(fontsize=text_size)

  # Set the threshold for different colors
  threshold = (cm.max() + cm.min()) / 2.

  # Plot the text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if norm:
      plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
    else:
      plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)

  # Save the figure to the current working directory
  if savefig:
    fig.savefig("confusion_matrix.png")

In [ ]:
make_confusion_matrix(y_labels,pred_classes,classes = class_names,figsize = (20,20))

* > From the confusion matrix we can clearly observe that the model is getting confused betwwen Melanocytic Nevi and Melanoma, Melanocyctic Nevi and Benign Kerotosis like Lesions, Tinea Ringworm Candidiasis and other Fungai Infections and Psoriasis pictures Lichen Planus and releated diseases.
* > In order to examine why our model is getting confused between the above mentioned diseases we can look at the data ourselves or consult a doctor to find out whethere these disesase can be classified properly just by looking at their images, often when it comes to skin diseases it cannot be classified properly just by looking at the image further testing is required.

**Let us see what our most wrong predictions are to understand more about our model's performance**

In [ ]:
filepaths = []
for filepath in test_data.list_files("./output/test/*/*.jpg",
                                     shuffle=False):
  filepaths.append(filepath.numpy())
filepaths[:10]

In [ ]:
len(filepaths)

In [ ]:
prediction_df = pd.DataFrame({"img_path": filepaths,
                        "y_true": y_labels,
                        "y_pred": pred_classes,
                        "pred_conf": pred_probs.max(axis=1), # get the maximum prediction probability value
                        "y_true_classname": [class_names[i] for i in y_labels],
                        "y_pred_classname": [class_names[i] for i in pred_classes]})
prediction_df.head()

In [ ]:
prediction_df["correct_pred"] = prediction_df["y_true"]==prediction_df["y_pred"]
prediction_df.head()

In [ ]:
top_50_wrong = prediction_df[prediction_df["correct_pred"] == False].sort_values("pred_conf", ascending=False)[:50]
top_50_wrong.head(10)

> As we saw in our confusion matrix Benign Keratosis-Like Lesions and Melanocyctic Nevi is often misclassified by our model.

**Conclusion**

> Transfer Learning architecture EfficientNetB5 works pretty well on the given dataset of 10 classes of skin diseases.But some of these diseases cannot be classified correctly just by looking at the images even by doctors so our model gets confused between different classes of diseases in our data. Currently it has a testing accuracy of 77% when trained for 21 epochs and it will improve if trained for another 10-15 epochs and it may get saturated around 80-85%.
Now if we train on certain distinguishable classes in our data instead of all the 10 classes the same EfficientNetB5 would perform extremely well.

**Note:**

> As I mentioned after feature extractor model training the training stopped even in fine tuned model around 21 epochs when it was supposed to train for 30 epochs and again it wasn't beacuse of earlystopping. My guess so for is my CPU ran out of memory in Kaggle and I am not sure why since I have implemented mixed_precision training. I generally work in Collab on all my projects and I have never had this problem before in Collab, am I bit new in implemented Deep Learning Models in Kaggle any suggestions to solve this problem and tips to make sure this doesn't happen in the future would greatly helpful.

**Thank You**